# TODO:
https://www.kaggle.com/competitions/playground-series-s3e2/discussion/378795
https://www.kaggle.com/competitions/playground-series-s3e2/discussion/378780

These show that we should incorporate original data but when validating using K fold methods, we should only validate based on data in the competition dataset and not on original dataset. So implement this technique for this competition

### Also, bagging resulted in a lot better score with simple mean in the last competition, thought it didn't score much better on public, it absolutely ranked up to 60th position in the final private LB. So,
## Remember to trust your cvs over pbl

# A Few more TODOs:
* select features with less than or equal to 10 values as catergorical features, instead of current 20, see if it improves the score
* Try target encoding, weights of evidence AND leave one out encoding, see which one performs better

# Imports

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import optuna
from sklearn.preprocessing import StandardScaler

from category_encoders import TargetEncoder, LeaveOneOutEncoder, WOEEncoder

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Loading Data

In [4]:
BASE_PATH = Path('../input/playground-series-s3e3')

# id is not going to be an informative feature, so we're dropping it for train
# but since we'll need test set's ids to make the submission file, so we'll save those in  a separate varible before dropping
train = pd.read_csv(BASE_PATH / "train.csv").drop(columns="id")
test = pd.read_csv(BASE_PATH / "test.csv")
test_idx = test.id
test = test.drop(columns="id")

# It's been shown that incorporating original data, improves scores - at least on the public leaderboard. So let's do that!
original = pd.read_csv('../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')

train.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,Male,...,80,1,10,2,3,10,0,7,8,0
1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,Male,...,80,1,4,3,3,4,2,0,3,0
2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,Male,...,80,2,4,3,3,3,2,1,2,0
3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,Female,...,80,0,15,1,1,6,0,0,2,0
4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,Female,...,80,0,31,0,3,31,14,4,10,1


In [5]:
original.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


# Pre-Pre-Processing

### Let's make the feature names and order consistent b/w our competition dataset and original dataset, before we concatenate

In [6]:
original['Attrition'] = (original['Attrition'] == 'Yes').astype(np.int64)

# in original data, id is termed as "EmployeeNumber", so let's drop it
original.drop(columns="EmployeeNumber", inplace=True)

In [7]:
# now reordering the features in original dataset
original = original[list(train.columns)]

### Someone in the first compeition showed that adding a source feature i.e. a feature that indicates whether a given record is from original dataset or synthetic improves performance, so let's' do that!

In [8]:
original["is_original"] = 1
train["is_original"] = 0
test["is_original"] = 0

### Let's finally concatenate

In [9]:
train_extended = pd.concat([train, original]).reset_index(drop=True)
len(train_extended)

3147

### checking for null values

In [11]:
pd.concat([train_extended.isnull().sum().rename("Missing in Train"),
           test.isnull().sum().rename("Missing in Test")], axis=1).sort_values(by="Missing in Train")

,Missing in Train,Missing in Test
Age,0,0.0
Over18,0,0.0
OverTime,0,0.0
PercentSalaryHike,0,0.0
PerformanceRating,0,0.0
RelationshipSatisfaction,0,0.0
StandardHours,0,0.0
NumCompaniesWorked,0,0.0
StockOptionLevel,0,0.0
TrainingTimesLastYear,0,0.0


#### Insights: No missing values! Something to celebrate! :p

## Let's also concatenate test data to train

In [12]:
y = train_extended.Attrition
y

0       0
1       0
2       0
3       0
4       1
       ..
3142    0
3143    0
3144    0
3145    0
3146    0
Name: Attrition, Length: 3147, dtype: int64

In [13]:
df = pd.concat([train_extended.drop(columns="Attrition"), test])

# Preprocessing

### Identifying Categorical Features

In [14]:
df.dtypes.sort_values()

Age                          int64
YearsSinceLastPromotion      int64
YearsInCurrentRole           int64
YearsAtCompany               int64
WorkLifeBalance              int64
TrainingTimesLastYear        int64
TotalWorkingYears            int64
StockOptionLevel             int64
StandardHours                int64
RelationshipSatisfaction     int64
PerformanceRating            int64
PercentSalaryHike            int64
NumCompaniesWorked           int64
MonthlyRate                  int64
YearsWithCurrManager         int64
MonthlyIncome                int64
JobSatisfaction              int64
DailyRate                    int64
DistanceFromHome             int64
Education                    int64
EmployeeCount                int64
HourlyRate                   int64
EnvironmentSatisfaction      int64
JobLevel                     int64
JobInvolvement               int64
is_original                  int64
Gender                      object
MaritalStatus               object
OverTime            

### Remember, being of type int, doesn't mean that the feature cannot be categorial.
#### Let's check for unique values in each column

In [15]:
df.nunique().sort_values()

StandardHours                  1
EmployeeCount                  1
Over18                         1
is_original                    2
PerformanceRating              2
OverTime                       2
Gender                         2
BusinessTravel                 3
Department                     3
MaritalStatus                  3
RelationshipSatisfaction       4
JobSatisfaction                4
WorkLifeBalance                4
StockOptionLevel               5
JobInvolvement                 5
EnvironmentSatisfaction        5
Education                      6
JobLevel                       6
EducationField                 6
TrainingTimesLastYear          7
JobRole                        9
NumCompaniesWorked            11
PercentSalaryHike             15
YearsSinceLastPromotion       16
YearsWithCurrManager          18
YearsInCurrentRole            19
DistanceFromHome              29
YearsAtCompany                38
TotalWorkingYears             41
Age                           43
HourlyRate

#### INSIGHTS: Taking a quick look at number of unique values in features reveals that we should be safe setting the threshold for to 20 unique values for what consitutes as a categorical feature
#### We'll drop columns with only one value as they bring nothing to the table

#### But feel free to use your own intuition and test & trial to figure our what's works best in terms of threshold and features

In [16]:
feats_to_drop = [col for col in df.columns if df[col].nunique()==1]
cat_features = [col for col in df.columns if df[col].nunique() <= 20 and df[col].nunique() > 1]

In [17]:
df.drop(columns=feats_to_drop, inplace=True)

#### We won't use one hot encoder here, because we already have a large ratio of features to rows and one hotting would increase that ratio by a large margin even further which will result in severe overfitting
#### Rather we'll use ordinal/label encoder (they're basically the same thing)

In [18]:
len(df), len(y)

(4266, 3147)

In [19]:
# but first let's separate test and train_extended
X_train = df.iloc[:-len(test), :]
X_test = df.iloc[-len(test): , :]

In [20]:
target_enc = TargetEncoder()
loo_enc = LeaveOneOutEncoder(sigma=0.05)
woe_enc = WOEEncoder(sigma=0.05)

target_enc.fit(X_train[cat_features], y)

X_train[cat_features] = target_enc.transform(X_train[cat_features])
X_test[cat_features] = target_enc.transform(X_test[cat_features])

X_train.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,is_original
0,36,0.223048,599,0.121711,24,3,0.124383,4,0.147996,42,...,2,1,10,2,3,10,0,7,8,0
1,35,0.128590,921,0.173391,8,3,0.103659,1,0.147996,46,...,4,1,4,3,3,4,2,0,3,0
2,32,0.128590,718,0.173391,26,3,0.196141,3,0.147996,80,...,4,2,4,3,3,3,2,1,2,0
3,38,0.128590,1488,0.121711,2,3,0.124383,3,0.124063,40,...,3,0,15,1,1,6,0,0,2,0
4,50,0.128590,1017,0.121711,5,4,0.124383,2,0.124063,37,...,3,0,31,0,3,31,14,4,10,0


In [22]:
numerical_feats = list(set(df.columns) - set(cat_features))

(len(numerical_feats) + len(cat_features)) == len(df.columns)

True

In [23]:
numerical_feats

['MonthlyIncome',
 'HourlyRate',
 'TotalWorkingYears',
 'Age',
 'DistanceFromHome',
 'DailyRate',
 'YearsAtCompany',
 'MonthlyRate']

## Always a good idea to scale the features

In [26]:
sc = StandardScaler()
X_train[numerical_feats] = sc.fit_transform(X_train[numerical_feats])
X_test[numerical_feats] = sc.transform(X_test[numerical_feats])

### Let's seprate comp and original sets

In [27]:
#let's also separate original and copetition data
X_comp = X_train[X_train.is_original==0]
y_comp = y[X_comp.index]

X_original = X_train[X_train.is_original==1]
y_original = y[X_original.index].reset_index(drop=True)
X_original = X_original.reset_index(drop=True)

# Modelling

### But first, let's setup cross validation

In [23]:
# for i, (x, y) in enumerate(zip([1,2,3], [4,5,6])):
#     print(f"{'*'*10} {i}")
#     print(f"X: {x}")
#     print(f"Y: {y}")    

In [24]:
# a = np.array([1,2,3])
# b = np.array([4,5,6])

# np.append(a, b)

In [33]:
# we're gonna train on the combined dataset but, we'll only calculate the validation score only on comp data

# N_FOLDS = 10

def cross_validate(X, y, model, model_verbose=None, verbose=None, X_original=None, y_original=None):
    N_FOLDS = 5
    all_scores = np.zeros(N_FOLDS)

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)

    
    for fold_id, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # for training we'll use data from both datasets
        if X_original is not None:
            X_tr = pd.concat([X_tr, X_original], axis=0).reset_index(drop=True)
            y_tr = pd.concat([y_tr, y_original], axis=0).reset_index(drop=True)
               
        model.fit(X_tr, y_tr, 
                  eval_set=[(X_val, y_val)],
                  early_stopping_rounds=50,
                 verbose=model_verbose)
        
        y_pred = model.predict_proba(X_val)[:, 1]
        
        auc = roc_auc_score(y_val, y_pred)
        
        print(f"Fold {fold_id} \t auc: {auc}")
        
        all_scores[fold_id] = (auc)
    
    avg_auc = np.mean(all_scores)
    
    print(f"Avg AUC: {avg_auc}")

In [34]:
# random params values - make sure to tune yours
xgb_params = {'n_estimators': 150,
                 'max_depth': 3,
                 'learning_rate': 0.1,
                 'min_child_weight': 4,
                 'subsample': 0.7,
                 'colsample_bytree': 0.3
             }


xgb_clf = xgb.XGBClassifier(**xgb_params)

cross_validate(X_comp, y_comp, xgb_clf, model_verbose=False,
                           X_original=X_original, y_original=y_original)

# xgb_clf.fit(X_train, y, verbose=0)

Fold 0 	 auc: 0.8356418918918919
Fold 1 	 auc: 0.9069256756756756
Fold 2 	 auc: 0.8169491525423729
Fold 3 	 auc: 0.8648305084745762
Fold 4 	 auc: 0.9005084745762711
Avg AUC: 0.8649711406321575


In [ ]:
# random params but feel free to tune
lgbm_params = {'n_estimators': 407,
                 'num_rounds': 274,
                 'learning_rate': 0.1,
                 'num_leaves': 195,
                 'max_depth': 9,
                 'min_data_in_leaf': 46,
                 'lambda_l1': 0.01,
                 'lambda_l2': 0.6,
                 'min_gain_to_split': 1.42,
                 'bagging_fraction': 0.45,
                 'feature_fraction': 0.3}


lgbm_clf = lgbm.LGBMClassifier(**lgbm_params)

cross_validate(X_train, y, lgbm_clf, model_verbose=-1)

# lgbm_clf.fit(X_train, y, verbose=False)

In [50]:
#random params but feel free to tune
catboost_params = {'loss_function': 'CrossEntropy',
                     'learning_rate': 0.76,
                     'l2_leaf_reg': 0.014,
                     'colsample_bylevel': 0.06,
                     'depth': 1,
                     'boosting_type': 'Plain',
                     'bootstrap_type': 'Bernoulli',
                     'min_data_in_leaf': 18,
                     'one_hot_max_size': 14,
                     'subsample': 0.99}

catboost_clf = catboost.CatBoostClassifier(**catboost_params)

cross_validate(X_comp, y_comp, catboost_clf, model_verbose=False)

# catboost_clf.fit(X_train, y, verbose=False)

Fold 0 	 auc: 0.8514358108108109
Fold 1 	 auc: 0.9086993243243243
Fold 2 	 auc: 0.8044915254237287
Fold 3 	 auc: 0.8661864406779661
Fold 4 	 auc: 0.9154237288135594
Avg AUC: 0.8692473660100777


## INSIGHTS:
let's use this method of cross validation to
* Tune all our models
* Select top k
* Take their predictions average
* submit

In [75]:
np.random.randint(1, 10, size=(2,3))

array([[7, 2, 9],
       [7, 1, 7]])

In [83]:
some_X = pd.DataFrame(data=np.random.randint(1, 10, size=(2,3)))
pd.concat([some_X, some_X], axis=0).reset_index(drop=True)

,0,1,2
0,6,1,9
1,3,3,5
2,6,1,9
3,3,3,5


# Hyperparameters Tuning

## XGBoost

In [35]:
def objective_xgb(trial, X, y, X_original, y_original):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 0.00001, 0.3),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0, step=0.05),
        'early_stopping_rounds': trial.suggest_int("early_stoppig_rounds", 40, 100)
    }
    # we're gonna train on the combined dataset but, we'll only calculate the validation score only on comp data

    N_FOLDS = 5
    all_scores = np.zeros(N_FOLDS)

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)

    for fold_id, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # for training we'll use data from both datasets
        if X_original is not None:
            X_tr = pd.concat([X_tr, X_original], axis=0)
            y_tr = pd.concat([y_tr, y_original], axis=0)
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        
        y_pred = model.predict_proba(X_val)[:, 1]
                
        auc = roc_auc_score(y_val, y_pred)        
        all_scores[fold_id] = auc
    
    avg_auc = np.mean(all_scores)
    
    print(f"Avg AUC: {avg_auc}")
    
    return avg_auc

In [36]:
study_xgb = optuna.create_study(study_name="xgboost_tuning", direction="maximize")
func = lambda trial: objective_xgb(trial, X_comp, y_comp, X_original, y_original)
study_xgb.optimize(func, n_trials=30)

[I 2023-01-21 08:16:17,618] A new study created in memory with name: xgboost_tuning
[I 2023-01-21 08:16:23,921] Trial 0 finished with value: 0.858027027027027 and parameters: {'n_estimators': 288, 'max_depth': 5, 'learning_rate': 0.025159214649555714, 'min_child_weight': 9, 'gamma': 0.0015314413576502184, 'subsample': 0.75, 'colsample_bytree': 0.45, 'early_stoppig_rounds': 64}. Best is trial 0 with value: 0.858027027027027.


Avg AUC: 0.858027027027027


[I 2023-01-21 08:16:26,818] Trial 1 finished with value: 0.8516515116811727 and parameters: {'n_estimators': 89, 'max_depth': 5, 'learning_rate': 0.053510979109602334, 'min_child_weight': 4, 'gamma': 0.006303644080049534, 'subsample': 0.6000000000000001, 'colsample_bytree': 0.8, 'early_stoppig_rounds': 90}. Best is trial 0 with value: 0.858027027027027.


Avg AUC: 0.8516515116811727


[I 2023-01-21 08:16:29,934] Trial 2 finished with value: 0.8550784470911589 and parameters: {'n_estimators': 166, 'max_depth': 3, 'learning_rate': 0.043210308054627326, 'min_child_weight': 1, 'gamma': 0.009719966460524504, 'subsample': 0.9000000000000001, 'colsample_bytree': 0.35000000000000003, 'early_stoppig_rounds': 58}. Best is trial 0 with value: 0.858027027027027.


...

Avg AUC: 0.8531299816765918


[I 2023-01-21 08:17:56,029] Trial 28 finished with value: 0.8544667315620705 and parameters: {'n_estimators': 234, 'max_depth': 6, 'learning_rate': 0.12000050538735188, 'min_child_weight': 3, 'gamma': 0.0001586766513144013, 'subsample': 0.7, 'colsample_bytree': 0.8, 'early_stoppig_rounds': 70}. Best is trial 13 with value: 0.8667127805771873.


Avg AUC: 0.8544667315620705


[I 2023-01-21 08:17:58,077] Trial 29 finished with value: 0.8538826729271645 and parameters: {'n_estimators': 110, 'max_depth': 3, 'learning_rate': 0.29844401791874936, 'min_child_weight': 6, 'gamma': 1.492083516589084e-05, 'subsample': 0.75, 'colsample_bytree': 0.4, 'early_stoppig_rounds': 93}. Best is trial 13 with value: 0.8667127805771873.


Avg AUC: 0.8538826729271645


In [37]:
study_xgb.best_value

0.8667127805771873

## INSIGHTS:
* With 10 fold cv and target encoding, we achieve a best value of 0.879. But i feel like 10 cv might be too much for such small dataset in that it's way more prone to randoness than a 5 fold is
* With 5 fold cv and target encoding, we achieve a best value of 0.87259 or 0.873

Although the pbL is just luck based casino game at this point, we'll still submit using the best params to make sure we're on the right path with such cross_validation technique. The last time i tried using this technique, it resulted in severe overfitting because idk how i somehow messed something up

In [38]:
study_xgb.best_params

{'n_estimators': 177,
 'max_depth': 3,
 'learning_rate': 0.2814017544513701,
 'min_child_weight': 8,
 'gamma': 0.00010877835071630076,
 'subsample': 0.75,
 'colsample_bytree': 0.2,
 'early_stoppig_rounds': 79}

In [39]:
xgb_params = {'n_estimators': 177,
             'max_depth': 3,
             'learning_rate': 0.2814,
             'min_child_weight': 8,
             'gamma': 0.0001,
             'subsample': 0.75,
             'colsample_bytree': 0.2,
             'early_stoppig_rounds': 79}

In [40]:
X_train_fr, X_val, y_train_fr, y_val = train_test_split(X_comp, y_comp, test_size=0.1, shuffle=True, random_state=1337,
                                                        stratify=y_comp)

In [41]:
X_train_fr = pd.concat([X_train_fr, X_original])
y_train_fr = pd.concat([y_train_fr, y_original])

In [42]:
xgb_tuned_clf = xgb.XGBClassifier(**xgb_params)
xgb_tuned_clf.fit(X_train_fr, y_train_fr, eval_set=[(X_val, y_val)], verbose=False)

[08:19:22] WARNING: ../src/learner.cc:627: 
Parameters: { "early_stoppig_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.2,
              early_stoppig_rounds=79, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=0.0001,
              gpu_id=-1, grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.2814, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
              min_child_weight=8, missing=nan, monotone_constraints='()',
              n_estimators=177, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, ...)

In [43]:
len(X_test) == len(test)

True

In [44]:
xgb_tuned_preds = xgb_tuned_clf.predict_proba(X_test)[:, 1]

In [46]:
# non-overfitting predictions
submission = pd.DataFrame({"id": test_idx, "Attrition": xgb_tuned_preds})
submission.head()

,id,Attrition
0,1677,0.093922
1,1678,0.051655
2,1679,0.040405
3,1680,0.133737
4,1681,0.819291


In [86]:
# # non-overfitting predictions
# submission = pd.DataFrame({"id": test_idx, "Attrition": xgb_tuned_preds})
# submission.head()

,id,Attrition
0,1677,0.196305
1,1678,0.043251
2,1679,0.010886
3,1680,0.082801
4,1681,0.132661


In [41]:
# # non-overfitting predictions
# submission = pd.DataFrame({"id": test_idx, "Attrition": xgb_tuned_preds})
# submission.head()

,id,Attrition
0,1677,0.401170
1,1678,0.117693
2,1679,0.000777
3,1680,0.015711
4,1681,0.295577


In [42]:
# OVERFITTED PREDICTIONS
# submission = pd.DataFrame({"id": test_idx, "Attrition": xgb_tuned_preds})
# submission.head()

,id,Attrition
0,1677,0.021201
1,1678,0.001126
2,1679,0.000017
3,1680,0.000405
4,1681,0.947035


In [47]:
submission.to_csv("submission.csv", index=False)